<a href="https://colab.research.google.com/github/UznetDev/Data-science-home-work/blob/main/27_Avg_2024_home_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SQL**

### Import librares

In [27]:
import pandas as pd

### Set-up for SQL

In [28]:
%load_ext sql
%sql sqlite://

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


##### Load dataset

In [29]:
monarchs = pd.read_csv('monarchs.csv')
presidents = pd.read_csv('presidents.csv')
prime_minister_terms = pd.read_csv('prime_minister_terms.csv')
prime_ministers = pd.read_csv('prime_ministers.csv')
states = pd.read_csv('states.csv')

##### Upload dataset to SQL

In [30]:
%sql DROP TABLE IF EXISTS monarchs;
%sql --persist monarchs

%sql DROP TABLE IF EXISTS presidents;
%sql --persist presidents

%sql DROP TABLE IF EXISTS prime_minister_terms;
%sql --persist prime_minister_terms

%sql DROP TABLE IF EXISTS prime_ministers;
%sql --persist prime_ministers

%sql DROP TABLE IF EXISTS states;
%sql --persist states

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.
 * sqlite://


'Persisted states'

### Class work

#### View all table

In [31]:
%%sql SELECT * FROM monarchs LIMIT 1;

 * sqlite://
Done.


index,country,continent,monarch
0,Brunei,Asia,Hassanal Bolkiah


In [32]:
%%sql SELECT * FROM presidents LIMIT 1;

 * sqlite://
Done.


index,country,continent,president
0,Egypt,Africa,Abdel Fattah el-Sisi


In [33]:
%%sql SELECT * FROM prime_minister_terms LIMIT 1;

 * sqlite://
Done.


index,prime_minister,pm_start
0,Mostafa Madbouly,2018


In [34]:
%%sql SELECT * FROM prime_ministers LIMIT 1;

 * sqlite://
Done.


index,country,continent,prime_minister
0,Egypt,Africa,Mostafa Madbouly


In [35]:
%%sql SELECT * FROM states LIMIT 1;

 * sqlite://
Done.


index,country,continent,indep_year
0,Australia,Oceania,1901


#### Subquiry & CTE (WITH)

##### Part 1 (*Subquiry*)

In [36]:
%%sql SELECT
    country,
    continent,
    (SELECT p1.prime_minister
     FROM prime_minister_terms AS p1
     INNER JOIN prime_ministers AS p2
     USING(prime_minister)
     WHERE p2.country = s.country
     ORDER BY pm_start
     LIMIT 1) AS prime_minister
FROM states AS s;

 * sqlite://
Done.


country,continent,prime_minister
Australia,Oceania,Scott Morrison
Brunei,Asia,Hassanal Bolkiah
Chile,South America,None
Egypt,Africa,Mostafa Madbouly
India,Asia,Narendra Modi
Norway,Europe,Jonas Gahr Støre
Oman,Asia,Haitham bin Tarik
Pakistan,Asia,Shehbaz Sharif
Portugal,Europe,António Costa
Uruguay,South America,None


In [37]:
%%sql SELECT country,
             continent,
             (SELECT MIN(pm_start)
             FROM prime_minister_terms AS p2
             WHERE p2.prime_minister = p1.prime_minister) AS prime_minister
FROM prime_ministers AS p1;

 * sqlite://
Done.


country,continent,prime_minister
Egypt,Africa,2018
Portugal,Europe,2015
Pakistan,Asia,2022
United Kingdom,Europe,2019
India,Asia,2014
Australia,Oceania,2018
Norway,Europe,2021
Brunei,Asia,1984
Oman,Asia,2020
New Zealand,Oceania,2017


In [38]:
%%sql SELECT country
FROM presidents AS p1
WHERE p1.president IN (SELECT prime_minister
                      FROM prime_ministers AS p2
                      WHERE p1.president=p2.prime_minister
                      AND p1.country=p2.country);

 * sqlite://
Done.


country


In [39]:
%%sql SELECT country
FROM prime_ministers AS p
WHERE (SELECT COUNT(*)
       FROM monarchs AS m
       WHERE m.country = p.country
       AND p.prime_minister = m.monarch);

 * sqlite://
Done.


country
Brunei
Oman


##### Part 2 (*CTE*)

In [40]:
%%sql
WITH a AS (SELECT 1 AS number)
SELECT number FROM a;

 * sqlite://
Done.


number
1


In [41]:
%%sql
WITH a AS (
    SELECT
        country,
        continent,
        (SELECT p1.prime_minister
         FROM prime_minister_terms AS p1
         INNER JOIN prime_ministers AS p2
         USING(prime_minister)
         WHERE p2.country = s.country
         ORDER BY pm_start
         LIMIT 1) AS prime_minister
    FROM states AS s
)
SELECT * FROM a;

 * sqlite://
Done.


country,continent,prime_minister
Australia,Oceania,Scott Morrison
Brunei,Asia,Hassanal Bolkiah
Chile,South America,None
Egypt,Africa,Mostafa Madbouly
India,Asia,Narendra Modi
Norway,Europe,Jonas Gahr Støre
Oman,Asia,Haitham bin Tarik
Pakistan,Asia,Shehbaz Sharif
Portugal,Europe,António Costa
Uruguay,South America,None


In [42]:
%%sql
WITH a AS (
    SELECT country,
             continent,
             (SELECT MIN(pm_start)
             FROM prime_minister_terms AS p2
             WHERE p2.prime_minister = p1.prime_minister) AS prime_minister
    FROM prime_ministers AS p1
)
SELECT * FROM a;

 * sqlite://
Done.


country,continent,prime_minister
Egypt,Africa,2018
Portugal,Europe,2015
Pakistan,Asia,2022
United Kingdom,Europe,2019
India,Asia,2014
Australia,Oceania,2018
Norway,Europe,2021
Brunei,Asia,1984
Oman,Asia,2020
New Zealand,Oceania,2017


In [43]:
%%sql
WITH a AS (
    SELECT country
    FROM presidents AS p1
    WHERE p1.president IN (SELECT prime_minister
                      FROM prime_ministers AS p2
                      WHERE p1.president=p2.prime_minister
                      AND p1.country=p2.country)
)
SELECT * FROM a;

 * sqlite://
Done.


country


In [44]:
%%sql
WITH a AS (
    SELECT country
    FROM prime_ministers AS p
    WHERE (SELECT COUNT(*)
           FROM monarchs AS m
           WHERE m.country = p.country
           AND p.prime_minister = m.monarch)
)
SELECT * FROM a;

 * sqlite://
Done.


country
Brunei
Oman


##### Result (*Home work*)

**Simple Subquery in SELECT**

Write a query to list all countries along with their continent and the earliest pm_start year for each country.
Use a simple subquery within the SELECT clause to find the earliest pm_start for each country.

In [45]:
%%sql SELECT country,
             continent,
             (SELECT MIN(pm_start)
             FROM prime_minister_terms AS p2
             WHERE p2.prime_minister = p1.prime_minister) AS prime_minister
FROM prime_ministers AS p1;

 * sqlite://
Done.


country,continent,prime_minister
Egypt,Africa,2018
Portugal,Europe,2015
Pakistan,Asia,2022
United Kingdom,Europe,2019
India,Asia,2014
Australia,Oceania,2018
Norway,Europe,2021
Brunei,Asia,1984
Oman,Asia,2020
New Zealand,Oceania,2017


**Simple Subquery in WHERE**

Write a query to list all countries where the president's name is the same as the prime minister's name.
Use a simple subquery within the WHERE clause to compare the names.

In [46]:
# 1 Simple Subquery
%%sql SELECT country
FROM presidents
WHERE president IN (
    SELECT prime_minister
    FROM prime_ministers
    WHERE country IN (SELECT country
                      FROM presidents)
);

 * sqlite://
Done.


country


In [47]:
# 2
%%sql SELECT country
FROM presidents AS p1
WHERE p1.president IN (SELECT prime_minister
                      FROM prime_ministers AS p2
                      WHERE p1.country=p2.country);

 * sqlite://
Done.


country


In [48]:
# 3
%%sql SELECT country
FROM presidents AS p1
WHERE (SELECT COUNT(*)
       FROM prime_ministers AS p2
       WHERE p1.president=p2.prime_minister
       AND p1.country=p2.country);

 * sqlite://
Done.


country


**Correlated Subquery in SELECT**

Write a query to list all countries along with their continent and the most recent pm_start year for each country.
Use a correlated subquery within the SELECT clause to find the most recent pm_start for each country.

In [49]:
%%sql SELECT country,
             continent,
             (SELECT MAX(pm_start)
             FROM prime_minister_terms AS p2
             WHERE p2.prime_minister = p1.prime_minister) AS prime_minister
FROM prime_ministers AS p1;

 * sqlite://
Done.


country,continent,prime_minister
Egypt,Africa,2018
Portugal,Europe,2015
Pakistan,Asia,2022
United Kingdom,Europe,2019
India,Asia,2014
Australia,Oceania,2018
Norway,Europe,2021
Brunei,Asia,1984
Oman,Asia,2020
New Zealand,Oceania,2017


**Correlated Subquery in WHERE**

Write a query to list all countries that have more than one prime minister who started their term after 2000.
Use a correlated subquery within the WHERE clause to filter countries based on the count of prime ministers.

In [50]:
%%sql SELECT country
FROM prime_ministers AS pm1
WHERE (
    SELECT COUNT(*)
    FROM prime_ministers AS pm2
    INNER JOIN prime_minister_terms AS pmt
    ON pm2.prime_minister = pmt.prime_minister
    WHERE pm2.country = pm1.country
    AND pmt.pm_start > '2000'
) > 1
GROUP BY country;

 * sqlite://
Done.


country


**Nested Subquery in SELECT**

Write a query to list all countries along with the name of the prime minister who started the earliest term in the continent that has the most countries.
Use nested subqueries in the SELECT clause to find the prime minister's name.

In [51]:
%%sql
SELECT country,
       continent,
       (SELECT prime_minister
        FROM prime_minister_terms AS pmt
        WHERE pmt.pm_start = (
            SELECT MIN(pm_start)
            FROM prime_minister_terms AS pmt2
            INNER JOIN prime_ministers AS pm
            ON pmt2.prime_minister = pm.prime_minister
            WHERE pmt2.prime_minister = pmt.prime_minister
            AND pm.country = s.country
        )) AS prime_minister
FROM states AS s
WHERE continent = (
    SELECT continent
    FROM states
    GROUP BY continent
    ORDER BY COUNT(*) DESC
    LIMIT 1
);

 * sqlite://
Done.


country,continent,prime_minister
Brunei,Asia,Hassanal Bolkiah
India,Asia,Narendra Modi
Oman,Asia,Haitham bin Tarik
Pakistan,Asia,Shehbaz Sharif


**Nested Subquery in WHERE**

Write a query to list all countries where the president's name matches the prime minister's name in the continent that has the highest average independence year.
Use nested subqueries in the WHERE clause to filter the countries.

In [72]:
# Mysql yokiy Postgresql uchun
# %%sql
# WITH mcha AS (
#     SELECT continent
#     FROM states
#     GROUP BY continent
#     ORDER BY AVG(indep_year) DESC
#     LIMIT 1
# )

# SELECT country, continent
# FROM states AS s
# WHERE continent IN mcha
# AND country in (
#     SELECT T2.country
#   FROM presidents AS p
#   INNER JOIN prime_ministers AS pm
#   ON MATCH(p.president) AGAINST(pm.prime_minister)
#   WHERE p.country = pm.country
#   AND p.country = s.country
# )

In [75]:
%%sql
WITH mcha AS (
    SELECT continent
    FROM states
    GROUP BY continent
    ORDER BY AVG(indep_year) DESC
    LIMIT 1
)

SELECT country, continent
FROM states AS s
WHERE continent IN mcha
AND country in (
    SELECT p.country
    FROM presidents AS p
    INNER JOIN prime_ministers AS pm
    ON p.country = pm.country
    WHERE p.president = pm.prime_minister
    AND p.country = s.country
)

 * sqlite://
Done.


country,continent
